In [1]:
# set to use cpu or gpu in the DQN
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "" # empty, so don't use gpu here

#### Import dependencies

In [2]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#### Set parameters

In [3]:
# set the environment, in application it would be the real (or virtual) trading trials
env = gym.make('CartPole-v0')

In [4]:
# decide the state and the actions size to model
# in this example, it's just 4 things to observe in each time period
# but in the appication, these are the number of observable features
# we will use to train the model 
state_size = env.observation_space.shape[0]
state_size

4

In [7]:
# the number of actions, also could be customized
action_size = env.action_space.n
action_size

2

#### Set batch size for the stochastic gradient descent

In [8]:
batch_size = 32

In [9]:
# number of trials for training
episodes = 1001

In [10]:
# store the output to a folder
output_dir = 'model_output/cartpole'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)